In [2]:
import pandas as pd
import os
from pybaseball import statcast, pitching_stats, batting_stats, playerid_lookup, statcast_batter, statcast_pitcher

from pull_data import *
%load_ext autoreload
%autoreload 2


In [3]:
from stat_miner import *

In [4]:
from rotowire import get_game_lineups

In [5]:
import pandas as pd
import requests
from bs4 import BeautifulSoup


url = "https://www.rotowire.com/baseball/daily-lineups.php"
soup = BeautifulSoup(requests.get(url).content, "html.parser")

data_pitching = []
data_batter = []
team_type = ''

for e in soup.select('.lineup__box ul li'):
    if team_type != e.parent.get('class')[-1]:
        order_count = 1
        team_type = e.parent.get('class')[-1]

    if e.get('class') and 'lineup__player-highlight' in e.get('class'):
        data_pitching.append({
            'date': e.find_previous('main').get('data-gamedate'),
            'game_time': e.find_previous('div', attrs={'class':'lineup__time'}).get_text(strip=True),
            'pitcher_name':e.a.get_text(strip=True),
            'team':e.find_previous('div', attrs={'class':team_type}).next.strip(),
            'lineup_throws':e.span.get_text(strip=True)
        })
    elif e.get('class') and 'lineup__player' in e.get('class'):
        data_batter.append({
            'date': e.find_previous('main').get('data-gamedate'),
            'game_time': e.find_previous('div', attrs={'class':'lineup__time'}).get_text(strip=True),
            'pitcher_name':e.a.get_text(strip=True),
            'team':e.find_previous('div', attrs={'class':team_type}).next.strip(),
            'pos': e.div.get_text(strip=True),
            'batting_order':order_count,
            'lineup_bats':e.span.get_text(strip=True)
        })
        order_count+=1

df_pitching = pd.DataFrame(data_pitching)
df_batter = pd.DataFrame(data_batter)

In [6]:
df_batter

,date,game_time,pitcher_name,team,pos,batting_order,lineup_bats
0,2025-06-03,6:40 PM ET,Jeremy Pena,Astros,SS,1,R
1,2025-06-03,6:40 PM ET,I. Paredes,Astros,3B,2,R
2,2025-06-03,6:40 PM ET,Jose Altuve,Astros,2B,3,R
3,2025-06-03,6:40 PM ET,C. Walker,Astros,1B,4,R
4,2025-06-03,6:40 PM ET,Yainer Diaz,Astros,DH,5,R
...,...,...,...,...,...,...,...
265,2025-06-03,10:10 PM ET,Will Smith,Dodgers,C,5,R
266,2025-06-03,10:10 PM ET,Max Muncy,Dodgers,3B,6,L
267,2025-06-03,10:10 PM ET,Andy Pages,Dodgers,CF,7,R
268,2025-06-03,10:10 PM ET,M. Conforto,Dodgers,LF,8,L


In [7]:
df_batter['team_id'] = df_batter['team'].map(get_team_mapping())

In [8]:
team_ids = df_batter['team_id'].unique()
team_ids = [int(team_id) for team_id in team_ids]
print(team_ids)

[117, 134, 115, 146, 112, 120, 114, 147, 143, 141, 108, 111, 158, 113, 109, 144, 140, 139, 116, 145, 118, 138, 110, 136, 135, 137, 142, 133, 121, 119]


In [79]:
all_player_ids = map_player_ids(team_ids)

No identically matched names found! Returning the 5 most similar names.
No identically matched names found! Returning the 5 most similar names.
No identically matched names found! Returning the 5 most similar names.
No identically matched names found! Returning the 5 most similar names.
No identically matched names found! Returning the 5 most similar names.
No identically matched names found! Returning the 5 most similar names.
No identically matched names found! Returning the 5 most similar names.
No identically matched names found! Returning the 5 most similar names.
No identically matched names found! Returning the 5 most similar names.
No identically matched names found! Returning the 5 most similar names.
No identically matched names found! Returning the 5 most similar names.
No identically matched names found! Returning the 5 most similar names.
No identically matched names found! Returning the 5 most similar names.
No identically matched names found! Returning the 5 most similar

                   name  player_id
0         Alec Burleson     676475
1        Andre Pallante     669467
2       Brendan Donovan     680977
3           Erick Fedde     607200
4          Iván Herrera     671056
..                  ...        ...
772         Tommy Edman     669242
773       Tony Gonsolin     664062
774          Will Klein     694361
775          Will Smith     519293
776  Yoshinobu Yamamoto     808967

[777 rows x 2 columns]
